In [ ]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from matplotlib.colors import hsv_to_rgb

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

1. Apply Hough transform (circular objects detection) and blob detection algorithm to detect coins present in image 'coins_1.jfif'. Comment on the outcomes of two methods.

In [ ]:
#Using Hough Transform (circular objects detection)
img = cv.imread('coins_1.jfif')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray = cv.GaussianBlur(gray, (7, 7), 1.5)

rows = gray.shape[0]
circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, rows/8,
                         param1 = 150, param2 = 25)

if circles is not None:
    circles = np.uint8(np.around(circles))
    for i in circles[0, :]:
        center = (i[0], i[1])
        # circle center
        cv.circle(img, center, 1, (0, 0, 200), 2)
        # circle outline
        radius = i[2]
        cv.circle(img, center, radius, (0, 255, 0), 2)
        
cv.imshow("detected circles", img)
cv.waitKey(0)

In [ ]:
#Using blob detection algorithm
img = cv.imread('coins_1.jfif')
img = cv.GaussianBlur(img, (3, 3), 0)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# parameter settings
params = cv.SimpleBlobDetector_Params()
# set up the filter
params.thresholdStep = 4

params.filterByColor = True
params.blobColor = 255
params.filterByArea = True
params.filterByConvexity = False
params.filterByInertia = False

# create object detector
detector = cv.SimpleBlobDetector_create(params)
keypoints = detector.detect(gray)


blank = np.zeros((0,0))
# draw keypoints
im_with_keypoints = cv.drawKeypoints(img, keypoints, blank, (0, 255, 0), 
                                    cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv.imshow('blobs', im_with_keypoints)
cv.waitKey(0)

2. Apply k-means clustering on 'zebra.jfif' to segment out the zebra.
<ul>
<li>You are required to determine the optimal k by plotting the within cluster sum of squares vs number of clusters (2-10).</li>
    <li>Apply the clustering method on 2 color space: BGR and HSV and compare the results obtained.</li>
</ul>

In [ ]:
img = cv.imread('zebra.jfif')
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
img_rgb = cv.GaussianBlur(img_rgb, (3, 3), 0)

plt.figure()
plt.imshow(img_rgb)
plt.show()

In [ ]:
img_reshape = img_rgb.reshape((-1, 3))
img_reshape = np.float32(img_reshape)

# k-means clustering
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 3 # K = 3 is the most suitable.
attempts = 10
ret, label, center = cv.kmeans(img_reshape, K, None, criteria, attempts, cv.KMEANS_PP_CENTERS)

# reshape into original dimensions
center = np.uint8(center)
res = center[label.flatten()]
result = res.reshape(img_rgb.shape)

from numpy import linalg as LN
white = (255, 255, 255)

Idx_min = np.argmin(LN.norm(center-white, axis = 1))
mask = result == center[Idx_min]
seg_img = img_rgb*(mask.astype("uint8"))

cv.imshow('segmentation', seg_img)
cv.waitKey(0)

In [ ]:
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

# reshape and convert to float32
img_reshape = img_hsv.reshape((-1, 3))
img_reshape = np.float32(img_reshape)

# Define criteria, attempts and number of clusters
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
attempts = 10
K = 3

ret, label, center = cv.kmeans(img_reshape, K, None, criteria, attempts, cv.KMEANS_PP_CENTERS)

# Convert the center to np.uint8
center = np.uint8(center)
res = center[label.flatten()]
result_img = res.reshape(img_hsv.shape)

In [ ]:
img_reshape_bgr = img.reshape((-1, 3))
img_reshape_bgr = np.float32(img_reshape_bgr)

# Define criteria, attempts and number of clusters
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
attempts = 10
K = 3

ret, label, center = cv.kmeans(img_reshape_bgr, K, None, criteria, attempts, cv.KMEANS_PP_CENTERS)

# Convert the center to np.uint8
center = np.uint8(center)
res = center[label.flatten()]
result_img_bgr = res.reshape(img.shape)

In [ ]:
cv.imshow('segmentation hsv', cv.cvtColor(result_img, cv.COLOR_HSV2RGB))
cv.imshow('segmentation bgr', cv.cvtColor(result_img_bgr, cv.COLOR_BGR2RGB))
cv.waitKey(0)

In [ ]:
plt.subplot(1, 3, 1)
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title('original'), plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(cv.cvtColor(result_img, cv.COLOR_HSV2RGB))
plt.title(f'k = {K} (HSV)'), plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(cv.cvtColor(result_img_bgr, cv.COLOR_BGR2RGB))
plt.title(f'k = {K} (BGR)'), plt.axis("off")
plt.show()